In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

import hasasia.sim as hsim

In [ ]:
SHAPE = 30
NREALS = 30
NFREQS = 40
NLOUDEST = 10

# Construct a pspace instance

In [ ]:
# construct a param_space instance, note that `nsamples` and `seed` don't matter here for how we'll use this
pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)

# get the parameter names from this library-space
param_names = pspace.param_names
num_pars = len(pspace.param_names)
print(f"{num_pars=} :: {param_names=}")

# choose each parameter to be half-way across the range provided by the library
pars = 0.5 * np.ones(num_pars)
# params = pars * pspace.param_samples
params = pspace.normalized_params(pars)
print(f"{pars=}")
print(f"{params=}")

# construct `sam` and `hard` instances based on these parameters,
# using otherwise all default parameters for this library
sam, hard = pspace.model_for_normalized_params(pars)
if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
    hard_name = 'Fixed Time'
elif isinstance(hard, holo.hardening.Hard_GW):
    hard_name = 'GW Only'
# sam, hard = pspace.model_for_params(pspace.normalized_params(pars)) #this is way slower, but why??

# run this model, retrieving binary parameters and the GWB
data = holo.librarian.run_model(sam, hard, NREALS, NFREQS, NLOUDEST, 
                                gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
print(f"retrieved data: {data.keys()=}")

In [ ]:
sspar = data['sspar']
bgpar = data['bgpar']

print(f"{sspar.shape=}")
print('%d out of %d' % (np.sum(sspar[3]<0), sspar[3].size))
print('%d out of %d' % (np.sum(bgpar[3]<0), bgpar[3].size))

In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_gw_edges=data['fobs_edges'], hard=holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR))
print('%d out of %d' % (np.sum(sspar[3]<0), sspar[3].size))
print('%d out of %d' % (np.sum(bgpar[3]<0), bgpar[3].size))